# Advanced DQM for a Drift Tube detector


## Intro
A novel approach for muons identification and track parameter estimation [[1]] consists in the implementation of an algorithm mixing artificial neural networks
and analytical methods on a FPGA. This algorithm is being tested on a
cosmic muon telescope at the Legnaro INFN National Laboratory (LNL), a detector composed by a
set of drift-tubes (DT) reproducing a small-scale replica of those in use at CMS. 

## Resources
- Description of the experimental setup -> [[2]] chapter 1 (1.2.1 explain the actual configuration) 
- How we track muons using scintillator signals -> [[2]] section 2.2, 2.4 + chapter 3
- How the ML trigger algorithm reconstructs tracks (mean timer technique) ->  + [[4]] (minimal summary) + [[3]] section 3.1 (complete documentation)

## Outline

1. Build a 2D dataset with t<sub>drift</sub> and the crossing angle &theta; as features
   * using scintillator signals
   * using the ML algorithm reconstruction hidden within data
2. Study the correlation between the two features 
3. Test the performance of "the algorithm" (we should find a name for that algorithm) using the 2D dataset built in 1.
   * NN architecture &rarr; more complex than 1x3x1 (maybe 2x3x3 or 2x5x5)
   * What is the average training time for the algorithm?
   * If we put a constraint on the crossing angle, does the algorithm detect the anomaly in the drift time distribution or it correctly sees the correlation between the two features?

[1]: https://arxiv.org/abs/2105.04428
[2]: http://tesi.cab.unipd.it/65910/1/Franceschetto_Giacomo.pdf
[3]: http://cds.cern.ch/record/1073687/files/NOTE2007_034.pdf?version=1
[4]: https://github.com/spiccinelli/LCP_projects_Y3/blob/track_group6/Project.ipynb


## Contacts
* Marco Zanetti <marco.zanetti@unipd.it>\n",
    "* Alberto Zucchetta <alberto.zucchetta@pd.infn.it>\n",
    "* Jacopo Pazzini <jacopo.pazzini@unipd.it>\n"